# Women's E-commerce Clothing Review

### Read the File

In [12]:
import pandas as pd
import numpy as np
import os
os.chdir('C:/Personal/71053934/Documents/SHAHRZAD/Python/Kaggle/Womens e-commerce clothing review')
df = pd.read_csv('Womens e-commerce review.csv')

In [15]:
df.drop(df[['Unnamed: 0']], axis = 1)

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits
8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses


In [16]:
df.dropna(inplace = True)

In [17]:
df['Recommended IND'].mean()

0.8181771945885464

### Split train and test data

In [18]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Review Text'], 
                                                    df['Recommended IND'], 
                                                    random_state=0)

In [19]:
print('X_train first entry:\n\n', X_train.iloc[0])
print('\n\nX_train shape: ', X_train.shape)

X_train first entry:

 Beautiful airy well made blouse. very flattering and feminine. i normally wear a size 8 and ordered a 10 per the other reviews and i am glad i did! the fit id perfect.


X_train shape:  (14746,)


### Count Vectorizer

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

# Fit the CountVectorizer to the training data
vect = CountVectorizer().fit(X_train)

In [23]:
vect.get_feature_names()[::1000]

['00',
 'aside',
 'cartwheels',
 'days',
 'exquisitely',
 'hardy',
 'laughs',
 'navigate',
 'pooches',
 'ruggedness',
 'spot',
 'touted',
 'wouldn']

In [24]:
len(vect.get_feature_names())

12146

In [25]:
# transform the documents in the training data to a document-term matrix
X_train_vectorized = vect.transform(X_train)

X_train_vectorized

<14746x12146 sparse matrix of type '<class 'numpy.int64'>'
	with 648955 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.linear_model import LogisticRegression

# Train the model
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
from sklearn.metrics import roc_auc_score

# Predict the transformed test documents
predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.788387055755


In [29]:
# get the feature names as numpy array
feature_names = np.array(vect.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))


Smallest Coefs:
['drab' 'weirdly' 'shame' 'poor' 'stretched' 'netting' 'sack' 'hopes'
 'additionally' 'unless']

Largest Coefs: 
['justice' 'test' 'compliments' 'volume' 'adore' 'classy' 'slimmer'
 'unique' 'pleasantly' 'add']


### Tfidf

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names())

4103

### Find area under the  curve for TfidVectorized training and test set

In [32]:
X_train_vectorized = vect.transform(X_train)
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predicted = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.788387055755


In [33]:
feature_names = np.array(vect.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))


Smallest Coefs:
['disappointed' 'wanted' 'cheap' 'returning' 'unflattering' 'was'
 'returned' 'huge' 'back' 'not']

Largest Coefs: 
['love' 'perfect' 'great' 'little' 'comfortable' 'with' 'soft'
 'compliments' 'fits' 'perfectly']


### n-grams

In [34]:
# Fit the CountVectorizer to the training data specifiying a minimum 
# document frequency of 5 and extracting 1-grams and 2-grams
vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

X_train_vectorized = vect.transform(X_train)

len(vect.get_feature_names())

26096

In [35]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.802382851211


In [36]:
feature_names = np.array(vect.get_feature_names())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['to love' 'returned' 'unflattering' 'not flattering' 'returning' 'cheap'
 'cute but' 'poor' 'shame' 'to like']

Largest Coefs: 
['unique' 'length was' 'perfect' 'would recommend' 'compliments' 'not too'
 'fun' 'beautifully' 'fit and' 'love this']
